# Printify API Integration Examples
This notebook demonstrates how to interact with the Printify API using GuzzleHttp for WooCommerce integration.

In [ ]:
require 'vendor/autoload.php';

use GuzzleHttp\Client;
use GuzzleHttp\Exception\GuzzleException;

## Client Setup
Initialize GuzzleHttp client with Printify API configuration

In [ ]:
$client = new Client([
    'base_uri' => 'https://api.printify.com/v1/',
    'headers' => [
        'Authorization' => 'Bearer ' . PRINTIFY_API_KEY,
        'Accept' => 'application/json',
    ]
]);

## Working with Shops
Examples of interacting with shop endpoints

In [ ]:
def get_shops():
    """Retrieve all shops associated with the account"""
    response = requests.get(f"{BASE_URL}/shops", headers=headers)
    return response.json()

def get_shop_details(shop_id):
    """Get detailed information about a specific shop"""
    response = requests.get(f"{BASE_URL}/shops/{shop_id}", headers=headers)
    return response.json()

## Product Import Methods
Methods for managing product synchronization

In [ ]:
function getAllProducts($client, $shopId) {
    try {
        $response = $client->get("shops/{$shopId}/products.json");
        return json_decode($response->getBody()->getContents(), true);
    } catch (GuzzleException $e) {
        throw new Exception('Failed to fetch products: ' . $e->getMessage());
    }
}

function importProductToWooCommerce($client, $shopId, $printifyProduct) {
    // Convert Printify product to WooCommerce format
    $product = [
        'name' => $printifyProduct['title'],
        'type' => 'variable',
        'description' => $printifyProduct['description'],
        'short_description' => $printifyProduct['description'],
        'status' => 'publish',
        'catalog_visibility' => 'visible',
        'meta_data' => [
            [
                'key' => '_printify_product_id',
                'value' => $printifyProduct['id']
            ],
            [
                'key' => '_printify_shop_id',
                'value' => $shopId
            ]
        ]
    ];

    // Create WooCommerce product
    return wc_create_product($product);
}

function syncProductVariants($client, $shopId, $productId, $variants) {
    foreach ($variants as $variant) {
        try {
            // Get detailed variant info
            $response = $client->get("shops/{$shopId}/products/{$productId}/variants/{$variant['id']}.json");
            $variantData = json_decode($response->getBody()->getContents(), true);

            // Create WooCommerce variant
            createProductVariant($productId, $variantData);
        } catch (GuzzleException $e) {
            error_log('Failed to sync variant: ' . $e->getMessage());
        }
    }
}

## Managing Products
Examples of product management operations

In [ ]:
def create_product(shop_id, product_data):
    """Create a new product in the specified shop"""
    response = requests.post(
        f"{BASE_URL}/shops/{shop_id}/products",
        headers=headers,
        json=product_data
    )
    return response.json()

def get_product(shop_id, product_id):
    """Retrieve a specific product"""
    response = requests.get(
        f"{BASE_URL}/shops/{shop_id}/products/{product_id}",
        headers=headers
    )
    return response.json()

## Order Management
Methods for handling order synchronization

In [ ]:
function createPrintifyOrder($client, $shopId, $wcOrder) {
    $orderData = [
        'external_id' => $wcOrder->get_id(),
        'line_items' => array_map(function($item) {
            return [
                'product_id' => get_post_meta($item->get_product_id(), '_printify_product_id', true),
                'variant_id' => get_post_meta($item->get_variation_id(), '_printify_variant_id', true),
                'quantity' => $item->get_quantity()
            ];
        }, $wcOrder->get_items()),
        'shipping_method' => 1,
        'shipping_address' => [
            'first_name' => $wcOrder->get_shipping_first_name(),
            'last_name' => $wcOrder->get_shipping_last_name(),
            'address1' => $wcOrder->get_shipping_address_1(),
            'address2' => $wcOrder->get_shipping_address_2(),
            'city' => $wcOrder->get_shipping_city(),
            'state' => $wcOrder->get_shipping_state(),
            'country' => $wcOrder->get_shipping_country(),
            'zip' => $wcOrder->get_shipping_postcode(),
            'email' => $wcOrder->get_billing_email(),
            'phone' => $wcOrder->get_billing_phone()
        ]
    ];

    try {
        $response = $client->post("shops/{$shopId}/orders.json", [
            'json' => $orderData
        ]);
        return json_decode($response->getBody()->getContents(), true);
    } catch (GuzzleException $e) {
        throw new Exception('Failed to create Printify order: ' . $e->getMessage());
    }
}

function syncOrderStatus($client, $shopId, $orderId, $status) {
    try {
        $response = $client->post("shops/{$shopId}/orders/{$orderId}/status.json", [
            'json' => ['status' => $status]
        ]);
        return json_decode($response->getBody()->getContents(), true);
    } catch (GuzzleException $e) {
        throw new Exception('Failed to update order status: ' . $e->getMessage());
    }
}

## Handling Orders
Examples of order management functionality

In [ ]:
def get_orders(shop_id):
    """Retrieve all orders for a shop"""
    response = requests.get(
        f"{BASE_URL}/shops/{shop_id}/orders",
        headers=headers
    )
    return response.json()

def create_order(shop_id, order_data):
    """Create a new order"""
    response = requests.post(
        f"{BASE_URL}/shops/{shop_id}/orders",
        headers=headers,
        json=order_data
    )
    return response.json()

## Shipping Profile Management
Methods for managing shipping profiles and rates

In [ ]:
function getShippingProfiles($client, $shopId) {
    try {
        $response = $client->get("shops/{$shopId}/shipping.json");
        return json_decode($response->getBody()->getContents(), true);
    } catch (GuzzleException $e) {
        throw new Exception('Failed to fetch shipping profiles: ' . $e->getMessage());
    }
}

function updateShippingProfile($client, $shopId, $profileId, $profileData) {
    try {
        $response = $client->put("shops/{$shopId}/shipping/{$profileId}.json", [
            'json' => $profileData
        ]);
        return json_decode($response->getBody()->getContents(), true);
    } catch (GuzzleException $e) {
        throw new Exception('Failed to update shipping profile: ' . $e->getMessage());
    }
}

## Webhooks Integration
Examples of webhook management

In [ ]:
def create_webhook(shop_id, webhook_data):
    """Create a new webhook subscription"""
    response = requests.post(
        f"{BASE_URL}/shops/{shop_id}/webhooks",
        headers=headers,
        json=webhook_data
    )
    return response.json()

def list_webhooks(shop_id):
    """List all webhooks for a shop"""
    response = requests.get(
        f"{BASE_URL}/shops/{shop_id}/webhooks",
        headers=headers
    )
    return response.json()

## Example Usage
Examples of using the above methods

In [ ]:
// Example: Sync all products
try {
    $products = getAllProducts($client, $shopId);
    foreach ($products as $product) {
        $wcProduct = importProductToWooCommerce($client, $shopId, $product);
        if ($wcProduct) {
            syncProductVariants($client, $shopId, $product['id'], $product['variants']);
        }
    }
} catch (Exception $e) {
    error_log('Product sync failed: ' . $e->getMessage());
}

// Example: Create Printify order from WooCommerce order
try {
    $wcOrder = wc_get_order(123);
    $printifyOrder = createPrintifyOrder($client, $shopId, $wcOrder);
    update_post_meta($wcOrder->get_id(), '_printify_order_id', $printifyOrder['id']);
} catch (Exception $e) {
    error_log('Order creation failed: ' . $e->getMessage());
}

In [ ]:
# Example: List all shops
shops = get_shops()
print("Shops:", json.dumps(shops, indent=2))

# Example: Create a webhook
webhook_data = {
    "topic": "order:created",
    "url": "https://your-site.com/webhook/printify"
}
# Uncomment to test with actual shop_id
# result = create_webhook("your_shop_id", webhook_data)
# print("Webhook created:", json.dumps(result, indent=2))

## Error Handling and Retries
Implementing robust error handling and retry logic

In [ ]:
function withRetry(callable $operation, $maxRetries = 3) {
    $attempts = 0;
    $lastException = null;

    while ($attempts < $maxRetries) {
        try {
            return $operation();
        } catch (GuzzleException $e) {
            $lastException = $e;
            $attempts++;
            
            if ($e->getCode() === 429) { // Rate limit hit
                $waitTime = $e->getResponse()->getHeaderLine('Retry-After') ?: 60;
                sleep((int)$waitTime);
                continue;
            }

            if ($attempts === $maxRetries) {
                throw $e;
            }

            sleep(pow(2, $attempts)); // Exponential backoff
        }
    }
    
    throw $lastException;
}

## Batch Processing
Processing large numbers of products efficiently

In [ ]:
function batchProcessProducts($client, $shopId, $batchSize = 10) {
    $page = 1;
    $processed = 0;

    do {
        $products = withRetry(function() use ($client, $shopId, $page) {
            $response = $client->get("shops/{$shopId}/products.json", [
                'query' => ['page' => $page, 'limit' => $batchSize]
            ]);
            return json_decode($response->getBody()->getContents(), true);
        });

        foreach ($products['data'] as $product) {
            withRetry(function() use ($client, $shopId, $product) {
                importProductToWooCommerce($client, $shopId, $product);
            });
            $processed++;
        }

        $page++;
        
    } while (count($products['data']) === $batchSize);

    return $processed;
}

## Secure Webhook Handling
Implementing webhook signature verification

In [ ]:
function verifyWebhookSignature($payload, $signature, $secret) {
    $calculatedSignature = hash_hmac('sha256', $payload, $secret);
    return hash_equals($calculatedSignature, $signature);
}

function handleWebhook() {
    $payload = file_get_contents('php://input');
    $signature = $_SERVER['HTTP_X_PRINTIFY_SIGNATURE'] ?? '';
    $secret = get_option('printify_webhook_secret');

    if (!verifyWebhookSignature($payload, $signature, $secret)) {
        http_response_code(401);
        die('Invalid signature');
    }

    $event = json_decode($payload, true);
    switch ($event['type']) {
        case 'order.created':
            handleOrderCreated($event['data']);
            break;
        case 'order.status_changed':
            handleOrderStatusChanged($event['data']);
            break;
    }

    http_response_code(200);
    echo json_encode(['status' => 'success']);
}

## Usage Examples
Examples of using the enhanced functionality

In [ ]:
// Example: Batch process products with retry logic
try {
    $processed = batchProcessProducts($client, $shopId);
    echo "Successfully processed {$processed} products\n";
} catch (Exception $e) {
    error_log('Batch processing failed: ' . $e->getMessage());
}

// Example: Use retry wrapper for API calls
try {
    $order = withRetry(function() use ($client, $shopId, $orderId) {
        return createPrintifyOrder($client, $shopId, $orderId);
    });
    echo "Order created successfully\n";
} catch (Exception $e) {
    error_log('Order creation failed after retries: ' . $e->getMessage());
}